In [ ]:
import numpy as np
import pandas as pd
from datetime import date
from random import seed
from random import random

import time
import scipy
import scipy.signal
import os, os.path
import matplotlib
import matplotlib.pyplot as plt

from pylab import imshow

# vgg16 model used for transfer learning on the dogs and cats dataset
from matplotlib import pyplot
# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# from keras.optimizers import SGD

from keras.layers import Conv2D
from keras.layers import MaxPooling2D

# from keras.optimizers import gradient_descent_v2
# SGD = gradient_descent_v2.SGD(...)

from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


import h5py
import sys
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc
# import NASA_plot_core.py as rcp

# Read Training Set Labels

In [ ]:
# plot dog photos from the dogs vs cats dataset
from matplotlib.image import imread
# define location of dataset
train_folder = '/Users/hn/Documents/01_research_data/dogs-vs-cats/train/'
test_folder = "/Users/hn/Documents/01_research_data/dogs-vs-cats/test1/"
# plot first few images
for i in range(9):
    # define subplot
    pyplot.subplot(330 + 1 + i)
    # define filename
    filename = train_folder + 'dog.' + str(i) + '.jpg'
    # load image pixels
    image = imread(filename)
    # plot raw pixel data
    pyplot.imshow(image)
# show the figure
pyplot.show()

In [ ]:
from keras.preprocessing.image import load_img
photo = load_img(train_folder + 'dog.' + str(0) + '.jpg', target_size=(200, 200))
print (photo.size)
photo

# Load prepaired data

In [ ]:
# load and confirm the shape
from numpy import load
photos = load(train_folder +'dogs_vs_cats_photos.npy')
labels = load(train_folder +'dogs_vs_cats_labels.npy')
print(photos.shape, labels.shape)

In [ ]:
# create directories
dataset_home = train_folder + 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']

for subdir in subdirs:
    # create label subdirectories
    labeldirs = ['dogs/', 'cats/']
    for labldir in labeldirs:
        newdir = dataset_home + subdir + labldir
        os.makedirs(newdir, exist_ok=True)

In [ ]:
# # organize dataset into a useful structure
# # seed random number generator
# seed(1)
# # define ratio of pictures to use for validation
# val_ratio = 0.25
# # copy training dataset images into subdirectories
# src_directory = train_folder
# for file in os.listdir(src_directory):
#     src = src_directory + '/' + file
#     dst_dir = 'train/'
#     if random() < val_ratio:
#         dst_dir = 'test/'
#     if file.startswith('cat'):
#         dst = dataset_home + dst_dir + 'cats/'  + file
#         shutil.copyfile(src, dst)
#     elif file.startswith('dog'):
#         dst = dataset_home + dst_dir + 'dogs/'  + file
#         shutil.copyfile(src, dst)

In [ ]:
# create directories
dataset_home = '/Users/hn/Documents/01_research_data/NASA/ML_data/train_images/' # 'finalize_single_vs_double/'

# create label subdirectories
labeldirs = ['single/', 'double/']
for labldir in labeldirs:
    newdir = dataset_home + labldir
    os.makedirs(newdir, exist_ok=True)
# copy training dataset images into subdirectories

src_directory = dataset_home
for file in listdir(src_directory):
    src = src_directory + '/' + file
    if file.startswith('single'):
        dst = dataset_home + 'single/'  + file
        shutil.copyfile(src, dst)
    elif file.startswith('double'):
        dst = dataset_home + 'double/'  + file
        shutil.copyfile(src, dst)

# Save Final Model

In [ ]:
# prepare iterator
# train_it = datagen.flow_from_directory('finalize_single_vs_double/',
#                                        class_mode='binary', 
#                                        batch_size=16,
#                                        target_size=(224, 224))

# fit model
# model.fit_generator(train_it, 
#                     steps_per_epoch=len(train_it), 
#                     epochs=10, verbose=1)

# save model
# model.save('final_model_single_vs_double.h5')

In [1]:
model_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/"

In [ ]:
# define cnn model
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))

    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# run the test harness for evaluating a model
def run_test_harness():
# define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]
    # prepare iterator
    train_it = datagen.flow_from_directory('finalize_dogs_vs_cats/',
        class_mode='binary', batch_size=16, target_size=(224, 224))
    # fit model
    model.fit(train_it, steps_per_epoch=len(train_it), epochs=10, verbose=0)
    # save model
    model.save(model_dir+'final_model_single_vs_double.h5')

# entry point, run the test harness
run_test_harness()

# Make Prediction

In [3]:
# Make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(224, 224))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 224, 224, 3)
    # center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img

# load an image and predict the class
def run_example():
    # load the image
    img = load_image('sample_image.jpg')
    # load model
    model = load_model('final_model.h5')
    # predict the class
    result = model.predict(img)
    print(result[0])

# entry point, run the example
run_example()